In [ ]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt

In [ ]:
!pip install konlpy

In [ ]:
# test dataset upload
from google.colab import files
myfile = files.upload()

In [ ]:
!unzip kordata.zip

In [ ]:
data = pd.read_csv('kordata.csv', encoding = 'cp949')

In [ ]:
data.head(3)

In [ ]:
data2 = list(data.document)
data2[0:2]

In [ ]:
##구두점 제거
from string import punctuation
def strip_pumctuation(s):
    return ''.join(c for c in s if c not in punctuation)

clean_title = []
for sent in data2 :
    clean = strip_pumctuation(sent)
    clean_title.append(clean)

clean_title[0:1]                 


In [ ]:
from konlpy.tag import Kkma
kkma = Kkma()

In [ ]:
import time
import datetime
start = time.time()

In [ ]:
dataset = []
for i in range(len(clean_title)) :
      dataset.append(kkma.nouns(re.sub('[^가-힣a-zA-Z\s]', '' , clean_title[i])))


sec = time.time()-start
times = str(datetime.timedelta(seconds=sec)),split(".")
times = times[0]
print(times)

In [ ]:
clean_title2 = []
for i in range(len(clean_title)) :
    clean_title2.append(re.sub('영화','',clean_title[i]))

clean_title3 = []
for i in range(len(clean_title2)) :
    clean_title2.append(re.sub('내용','',clean_title2[i]))

In [ ]:
!pip install gensim
from gensim import corpora, models
import gensim

In [ ]:
high_score_reviews = dataset

In [ ]:
high_score_reviews = [[y for y in x if not len(y)==1]
                      for x in high_score_reviews]
dictionary = corpora.Dictionary(high_score_reviews)
corpus = [dictionary.doc2bow(text) for text in high_score_reviews]

In [ ]:
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel

coherence_values = []
for i in range(2,15) :
    Idamodel = gensim.models.Idamodel.LdaModel(corpus, num_topics = i, id2word = dictionary)
    coherence_model_Ida = CoherenceModel(model = Idamodel, texts = high_score_reviews, dictionary, topn=10)
    coherence_Ida = coherence_model_Ida.get_coherence()
    coherence_values.append(coherence_Ida)

In [ ]:
x = range(2,15)
plt.plot(x,coherence_values)
plt.xlabel("number of topics")
plt.ylabel("coherence score")
plt.show()          

In [ ]:
import matplotlib.pyplot as plt
perplexity_values = []
for i in range(2,20) :
    Idamodel = gensim.models.Idamodel.LdaModel(corpus, num_topics=i, id2word=dictionary)
    perplexity_values.append(Idamodel.log_perplexity(corpus))
    

In [ ]:
x=range(2,20)
plt.plot(x,perplexity_values)
plt.xlabel("number of topics")
plt.ylabel("perplexity score")
plt.show()      

In [ ]:
# 결과 바탕으로 갯수 수정
Idamodel = gensim.models.Idamodel.LdaModel(corpus, num_topics=6, alpha=0.1,
                                           id2word=dictionary)

In [ ]:
Idamodel.print_topics(num_words=20)

In [ ]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(Idamodel, corpus, dictionary)
vis